# ELSER Excel → Elasticsearch (Semantic Search) — One-Click Notebook

This notebook does two things:
1. **(Optional)** Ingest an Excel/CSV file into an Elasticsearch index using the **ELSER** ingest pipeline (`ml.tokens`).
2. **Run a semantic search** over that index and save a **client-friendly HTML report** — so you can reopen and show results **without rerunning** a live query.

> **Tip:** Set `RUN_INGEST=False` if your index already contains the Excel data (to avoid reindexing).


#### 0) Install Python packages (first run only)

In [ ]:

# If these are already installed in your venv, you can skip this cell.
# In VS Code/Jupyter on your machine, uncomment the next line and run once.
#!pip install elasticsearch==8.14.0 "urllib3<2" pandas openpyxl python-dateutil

#### 1) Configuration

In [ ]:

import os, json, time
from pathlib import Path
import pandas as pd
from datetime import datetime
from dateutil import parser as dtparser
from elasticsearch import Elasticsearch, helpers

# ---- Elasticsearch / ELSER ----
ES_URL  = os.environ.get("ES_URL",  "http://localhost:9200")
ES_USER = os.environ.get("ES_USER", "elastic")
ES_PASS = os.environ.get("ES_PASS", "changeme")

MODEL_ID     = ".elser_model_2_linux-x86_64"
PIPELINE_ID  = "elser_v2_pipeline"
TOKENS_FIELD = "ml.tokens"

# ---- Excel source ----
INDEX_NAME  = "excel_elser_index" 
FILE_PATH   = r"C:\Users\dell\elser-python\long_distance_runners_record.xlsx"  
SHEET_NAME  = "Sheet1"  

ID_COL       = "Runner ID"
TITLE_COL    = "Name"
EVENT_COL    = "Event"              
COUNTRY_COL  = "Country"
DATE_COL     = "Date"
TIME_COL     = "Time (HH:MM:SS)"
POSITION_COL = "Position"
UPDATED_COL  = "updated_at"         

# ---- Search ----
QUERY_TEXT = "fast half marathon runners from Kenya"
TOPK       = 5

RUN_INGEST = True

## 2) Helpers

In [ ]:
def wait_es(es, timeout_s=60):
    deadline = time.time() + timeout_s
    while time.time() < deadline:
        try:
            es.info()
            return
        except Exception:
            time.sleep(1)
    raise RuntimeError("Elasticsearch not responding")

def ensure_model_started(es):
    try:
        stats = es.ml.get_trained_models_stats(model_id=MODEL_ID)
        tms = stats.get("trained_model_stats", [])
        if tms:
            dstats = tms[0].get("deployment_stats") or {}
            if dstats.get("state") == "started":
                return
    except Exception:
        pass
    try:
        es.ml.start_trained_model_deployment(
            model_id=MODEL_ID,
            number_of_allocations=1,
            threads_per_allocation=1,
            queue_capacity=1024,
        )
    except Exception:
        pass

def ensure_pipeline(es):
    pipeline = {
        "processors": [
            {
                "inference": {
                    "model_id": MODEL_ID,
                    "input_output": [
                        {"input_field": "content", "output_field": TOKENS_FIELD}
                    ],
                    "inference_config": {"text_expansion": {}}
                }
            }
        ]
    }
    es.ingest.put_pipeline(id=PIPELINE_ID, processors=pipeline["processors"])

def ensure_index(es, index: str, with_extra_fields=None):
    if es.indices.exists(index=index):
        return
    props = {
        "content": {"type": "text"},
        "ml": {"properties": {"tokens": {"type": "rank_features"}}}
    }
    if with_extra_fields:
        props.update(with_extra_fields)
    es.indices.create(index=index, body={"mappings": {"properties": props}})

def to_dt(v):
    if pd.isna(v):
        return None
    if isinstance(v, datetime):
        return v
    try:
        return dtparser.parse(str(v))
    except Exception:
        return None

#### 3) Ingest Excel into ES (ELSER pipeline)

In [ ]:
def ingest_excel(es, index: str, file_path: Path, sheet=None,
                 id_col="id", title_col="title", event_col="body",
                 country_col="Country", date_col="Date", time_col="Time (HH:MM:SS)",
                 position_col="Position", updated_col="updated_at", batch=1000):
    file_path = Path(file_path)
    if file_path.suffix.lower() == ".xlsx":
        sheet_name = None
        if sheet is not None:
            try:
                sheet_name = int(sheet)
            except ValueError:
                sheet_name = sheet
        df = pd.read_excel(file_path, sheet_name=sheet_name, engine="openpyxl")
    elif file_path.suffix.lower() == ".csv":
        df = pd.read_csv(file_path)
    else:
        raise SystemExit("Unsupported tabular format. Use .xlsx or .csv")

    cols = {c.lower().strip(): c for c in df.columns}
    def col(name): return cols.get(name.lower(), name)

    id_col       = col(id_col)
    title_col    = col(title_col)
    event_col    = col(event_col)
    country_col  = col(country_col)
    date_col     = col(date_col)
    time_col     = col(time_col)
    position_col = col(position_col)
    updated_col  = col(updated_col)

    missing = [c for c in [id_col, title_col] if c not in df.columns]
    if missing:
        raise SystemExit(f"Missing required columns: {missing}")

    ensure_index(es, index, with_extra_fields={
        "id":        {"type": "keyword"},
        "title":     {"type": "text"},
        "body":      {"type": "text"},
        "country":   {"type": "keyword"},
        "event":     {"type": "keyword"},
        "date":      {"type": "date"},
        "time_raw":  {"type": "keyword"},
        "position":  {"type": "integer"},
        "updated_at":{"type": "date"}
    })

    actions = []
    for _, row in df.iterrows():
        rid      = row.get(id_col)
        title    = row.get(title_col)
        event    = row.get(event_col) if event_col in df.columns else None
        country  = row.get(country_col) if country_col in df.columns else None
        date_val = row.get(date_col) if date_col in df.columns else None
        time_val = row.get(time_col) if time_col in df.columns else None
        pos_val  = row.get(position_col) if position_col in df.columns else None
        updated  = to_dt(row.get(updated_col)) if updated_col in df.columns else None

        date_iso = None
        if date_val is not None:
            try:
                date_iso = to_dt(date_val).date().isoformat()
            except Exception:
                pass

        parts = []
        if title:    parts.append(str(title))
        if event:    parts.append(f"Event: {event}")
        if country:  parts.append(f"Country: {country}")
        if date_iso: parts.append(f"Date: {date_iso}")
        if time_val not in (None, ""): parts.append(f"Time: {time_val}")
        if pos_val not in (None, ""):  parts.append(f"Position: {pos_val}")
        content = ". ".join(parts) if parts else (str(title) or "")

        doc = {
            "id":       rid,
            "title":    title,
            "body":     event,
            "event":    event,
            "country":  country,
            "date":     date_iso,
            "time_raw": str(time_val) if time_val is not None else None,
            "position": int(pos_val) if str(pos_val).isdigit() else None,
            "content":  content,
        }
        if updated is not None:
            doc["updated_at"] = updated.isoformat()

        actions.append({
            "_op_type": "index",
            "_index": index,
            "_id": str(rid) if rid is not None else None,
            "pipeline": PIPELINE_ID,
            "_source": doc
        })

    if not actions:
        print("No rows to index."); return

    print(f"Indexing {len(actions)} rows from '{file_path.name}' → '{index}' via '{PIPELINE_ID}'...")
    success, fail = helpers.bulk(es, actions, stats_only=True, chunk_size=batch, request_timeout=120)
    es.indices.refresh(index=index)
    print(f"Done. success={success}, failed={fail}")

#### 4) Connect and (optionally) ingest

In [ ]:
ES = Elasticsearch(ES_URL, basic_auth=(ES_USER, ES_PASS), request_timeout=120)

wait_es(ES)
ensure_model_started(ES)
ensure_pipeline(ES)

if RUN_INGEST:
    ingest_excel(
        ES, INDEX_NAME, FILE_PATH, SHEET_NAME,
        id_col=ID_COL, title_col=TITLE_COL, event_col=EVENT_COL,
        country_col=COUNTRY_COL, date_col=DATE_COL, time_col=TIME_COL,
        position_col=POSITION_COL, updated_col=UPDATED_COL
    )
else:
    print("RUN_INGEST=False — skipping ingest.")


Indexing 5 rows from 'long_distance_runners_record.xlsx' → 'excel_elser_index' via 'elser_v2_pipeline'...


C:\Users\dell\AppData\Local\Temp\ipykernel_13364\212501029.py:100: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, fail = helpers.bulk(es, actions, stats_only=True, chunk_size=batch, request_timeout=120)


Done. success=5, failed=0


#### 5) Run semantic search and save a report

In [ ]:
from IPython.display import display, HTML
import pandas as pd
import json
from pathlib import Path

body = {
    "size": TOPK,
    "query": {
        "text_expansion": {
            TOKENS_FIELD: {
                "model_id": MODEL_ID,
                "model_text": QUERY_TEXT
            }
        }
    },
    "_source": ["id","title","event","country","content","updated_at"]
}
res = ES.search(index=INDEX_NAME, body=body)

hits = res.get("hits", {}).get("hits", [])
rows = []
for h in hits:
    s = h["_source"]
    rows.append({
        "score": round(h["_score"], 3),
        "id": s.get("id"),
        "name/title": s.get("title"),
        "event": s.get("event"),
        "country": s.get("country"),
        "snippet": (s.get("content") or "")[:180].replace("\n"," "),
        "updated_at": s.get("updated_at")
    })

df = pd.DataFrame(rows)
display(df)

cache_path = Path("elser_cached_results.json")
cache_path.write_text(json.dumps({"query": QUERY_TEXT, "index": INDEX_NAME, "hits": rows}, indent=2), encoding="utf-8")
print(f"Cached results → {cache_path.resolve()}")

html_rows = "".join(
    f"<tr><td>{r['score']}</td><td>{r['name/title']}</td><td>{r['event']}</td><td>{r['country']}</td><td>{r['snippet']}</td></tr>"
    for r in rows
)
report_html = f"""
<!doctype html>
<html><head><meta charset="utf-8"><title>ELSER Search Report</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; margin: 20px; }}
h1 {{ margin-bottom: 0; }}
small {{ color: #666; }}
table {{ border-collapse: collapse; width: 100%; margin-top: 12px; }}
th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
th {{ background: #f4f4f4; }}
</style></head>
<body>
  <h1>ELSER Search Report</h1>
  <small>Index: <b>{INDEX_NAME}</b> • Query: <b>{QUERY_TEXT}</b> • Hits: <b>{len(rows)}</b></small>
  <table>
    <thead><tr><th>Score</th><th>Name/Title</th><th>Event</th><th>Country</th><th>Snippet</th></tr></thead>
    <tbody>{html_rows}</tbody>
  </table>
</body></html>
"""
report_path = Path("elser_search_report.html")
report_path.write_text(report_html, encoding="utf-8")
print(f"Saved HTML report → {report_path.resolve()}")
display(HTML(report_html))


,score,id,name/title,event,country,snippet,updated_at
0,24.450,5,David Lee,Half Marathon,Kenya,David Lee. Event: Half Marathon. Country: Keny...,None
1,13.424,2,Alice Smith,Half Marathon,UK,Alice Smith. Event: Half Marathon. Country: UK...,None
2,10.521,4,Sophia Johnson,Marathon,Ethiopia,Sophia Johnson. Event: Marathon. Country: Ethi...,None
3,8.928,1,John Doe,Marathon,USA,John Doe. Event: Marathon. Country: USA. Date:...,None
4,3.790,3,Michael Brown,10K,Canada,Michael Brown. Event: 10K. Country: Canada. Da...,None


Cached results → C:\Users\dell\Downloads\elser_cached_results.json
Saved HTML report → C:\Users\dell\Downloads\elser_search_report.html


Score,Name/Title,Event,Country,Snippet
24.45,David Lee,Half Marathon,Kenya,David Lee. Event: Half Marathon. Country: Kenya. Date: 2025-05-01. Time: 01:02:55. Position: 1
13.424,Alice Smith,Half Marathon,UK,Alice Smith. Event: Half Marathon. Country: UK. Date: 2025-02-10. Time: 01:05:30. Position: 2
10.521,Sophia Johnson,Marathon,Ethiopia,Sophia Johnson. Event: Marathon. Country: Ethiopia. Date: 2025-04-12. Time: 02:08:10. Position: 3
8.928,John Doe,Marathon,USA,John Doe. Event: Marathon. Country: USA. Date: 2025-01-15. Time: 02:12:45. Position: 5
3.79,Michael Brown,10K,Canada,Michael Brown. Event: 10K. Country: Canada. Date: 2025-03-05. Time: 00:32:15. Position: 1
